<head><b><center><h1>TALLER N°2 DE CIENCIA DE DATOS</h1></center>
    <center>Arbol de Clasificacion</center></b>
</head>

<h3>Integrantes:</h3>
<ul>
    <li>Cristian Montiel</li>
    <li>Mathias Solar</li>
</ul>
    

<h3><b>Actividad 1: Organización del Set de Datos</b></h3>

En la actividad 1, se ha optado por utilizar la estrategia de validación cruzada para la evaluación de los parámetros de los modelos de clasificación. Esta elección se fundamenta en diversas razones que contribuyen a una evaluación más robusta y confiable del rendimiento del modelo.

En primer lugar, la validación cruzada permite una evaluación más robusta al utilizar múltiples particiones de datos para entrenamiento y prueba. Esto reduce la posibilidad de sesgo en el rendimiento del modelo causado por una única división de datos. Además, al realizar k entrenamientos y pruebas con diferentes conjuntos de datos, se minimiza el impacto de la variabilidad en los datos, siendo crucial para conjuntos de datos limitados o cuando se busca la generalización del modelo.

Además, la validación cruzada garantiza que cada observación en el conjunto de datos tenga la oportunidad de formar parte del conjunto de prueba en al menos una iteración, asegurando la consideración de todas las instancias en la evaluación del modelo.

In [1]:
import numpy as np
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix, classification_report

# Leer el archivo de muestra
with open("muestra_taller_2_entrenamiento.csv", "r") as file:
    muestra = [line.strip().split(",") for line in file.readlines()]

# Convertir los valores objetivo a 0 o 1
muestra = np.array(muestra, dtype='object')

muestra[:, -1] = np.where(muestra[:, -1] == "0.0", 0, 1)  #(condicion , aplicacion si se cumple, else)

# Obtener valores de características y objetivo
X = muestra[:, :-1].astype(float) #selecciona todas las columnas de cada fila pero no la ultima columna
y = muestra[:, -1].astype(float) #selecciona la ultima columna de cada fila

# Instanciar el clasificador con la seed 1234
clf = DecisionTreeClassifier(random_state=1234)

# Usar 5 carpetas de validación cruzada para evaluar el clasificador
cv_results = cross_validate(clf, X, y, cv=5)

<h3><b>Actividad 2: : Selección de criterio de evaluación</b></h3>

Para el desarrallo de la actividad , nuestra decision consitio en la eleccion de la metrica de evaluacion correspondiente a <b>F1 score</b>. Su eleccion se debe a la tasa de exito (vease a encontrar los casos de fraude) sin que hubiesen casos fallidos (encontrar casos normales),ademas de ser equilibrado a la ahora de mostrar casos que son falsamente exitosos y fallidos a comparacion de tecnicas como recall (mayor cantidad de casos de falsos exitosos).

Para poder hacer uso de dicha metrica , se necesito de la yuda de la libreria sklearn.metrics, que ademas de proporcionar la metrica necesaria tambien incluye otras como recall o precision.La funcion que se proporciona se denomina f1_score.

Esta métrica la utilizaremos en la función que realiza la busqueda de la mejor grilla, además de calcularla individualmente para determinar si nuestro modelo de clasificación es lo suficientemente bueno o no. El que un modelo sea bueno o malo se define como la capacidad para identificar transferencias fraudulentas como legítimas.

In [2]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from matplotlib import pyplot as plt
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Definir los parámetros de búsqueda de la grilla
param_grid = {
    'max_depth': [1, 2, 3, 4, 5, None],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 3, 4, 5, 10],
    'splitter': ['best', 'random'],
    'random_state':[123,99,43,12]
}

# Instanciar el clasificador
clf = DecisionTreeClassifier(random_state =123)

# Realizar búsqueda de grilla con validación cruzada de 5 carpetas
random_search = RandomizedSearchCV(clf, param_grid, scoring='f1', n_iter=50, cv=5, random_state=1234)

# Ajustar el modelo con búsqueda de grilla
random_search.fit(X_train, y_train)

# Imprimir mejores parámetros y mejor puntaje
print("Mejores parámetros: ", random_search.best_params_)
print("Mejor puntaje: ", random_search.best_score_)

# Crear el clasificador con los mejores parámetros
best_params = random_search.best_params_
clf = DecisionTreeClassifier(random_state=random_search.best_params_['random_state'],
                            max_depth=random_search.best_params_['max_depth'],
                            criterion=random_search.best_params_['criterion'],
                            min_samples_split=random_search.best_params_['min_samples_split'],
                            splitter=random_search.best_params_['splitter'])

# Entrenar el modelo con los datos de entrenamiento
clf.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_hat = clf.predict(X_test)

#Se calcula el f1_score para nuestro módelo
f1 = f1_score(y_test, y_hat)
print("F1 Score: {:.3f}".format(f1))

print("Informe de clasificación:")
print(classification_report(y_test, y_hat))

accuracy = accuracy_score(y_test, y_hat)
precision = precision_score(y_test, y_hat)
recall = recall_score(y_test, y_hat)
f1 = f1_score(y_test, y_hat)
confusion_mat = confusion_matrix(y_test, y_hat)

# Imprimir los mejores parámetros y las métricas de evaluación
print("Accuracy: {:.3f}".format(accuracy))
print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("F1 Score: {:.3f}".format(f1))
print("Matriz de confusión:")
print(confusion_mat)

Mejores parámetros:  {'splitter': 'best', 'random_state': 99, 'min_samples_split': 2, 'max_depth': 4, 'criterion': 'gini'}
Mejor puntaje:  0.922483289755095
F1 Score: 0.926
Informe de clasificación:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1903
         1.0       0.98      0.88      0.93        72

    accuracy                           0.99      1975
   macro avg       0.99      0.94      0.96      1975
weighted avg       0.99      0.99      0.99      1975

Accuracy: 0.995
Precision: 0.984
Recall: 0.875
F1 Score: 0.926
Matriz de confusión:
[[1902    1]
 [   9   63]]


<h3><b>Actividad 3: Selección de modelos e hiper-parámetros</b></h3>

Se seleccionaron dos modelos adecuados para abordar el problema de detección de fraudes: modelo de árboles de decisión y el modelo no supervisado de clustering. 
Se escogio el arbol de claisificacion porque su versatilidad y facilidad de interpretación los hacen ideales. Dos parámetros clave a ajustar son la profundidad del árbol, que controla su complejidad, y la métrica de selección de características, como la entropia o la indice de Gini.

tambien el aprendizaje supervisado ya que estos redicen una variable objetivo a partir de un conjunto de datos de características. Son versátiles y se adaptan a diversas aplicaciones, incluida la detección de fraudes. Los parámetros ajustables incluyen el método de optimización, como el descenso del gradiente, y el criterio de parada, que determina cuándo se detiene el ajuste del modelo.

para la busqueda del mejor clasificador La estrategia elegida es la validación cruzada con 5 carpetas para evaluar el rendimiento del clasificador en un conjunto independiente. Se realizarán dos experimentos para cada tipo de modelo:

Árbol de Decisión (Experimento 1): Profundidad del árbol de 5 niveles y métrica de selección de características de información mutua.

Árbol de Decisión (Experimento 2): Profundidad del árbol de 10 niveles y métrica de selección de características de ganancia de Gini.

In [3]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif as info_mutual #es una funcion

# Leer el archivo de muestra
with open("muestra_taller_2_entrenamiento.csv", "r") as file:
    muestra = [line.strip().split(",") for line in file.readlines()]

# Convertir los valores objetivo a 0 o 1
muestra = np.array(muestra, dtype='object')
muestra[:, -1] = np.where(muestra[:, -1] == "0.0", 0, 1)

# Obtener valores de características y objetivo
X = muestra[:, :-1].astype(float)
y = muestra[:, -1].astype(float)

# Experimento 1: Árbol de decisión con profundidad de 5 niveles y métrica de criterio entropy
clf_svm = svm.SVC(kernel='poly') 
clf_svm.fit(X_train, y_train)
y_predicho = clf_svm.predict(X_test)
precision_svm = accuracy_score(y_test, y_predicho)
print("Exactitud del clasificador SVM: {:.2f}".format(precision_svm))

clf_1 = DecisionTreeClassifier(max_depth=5, criterion="entropy") #-> criterion =>{'log_loss', 'gini', 'entropy'}
cv_results_1 = cross_validate(clf_1, X, y, cv=5)

# Utilizar SelectFromModel para aplicar la métrica de selección de características
selector = SelectFromModel(clf_1)
X_selected = selector.fit_transform(X, y)

# Imprimir los resultados de los experimentos
print("Experimento 1: Árbol de decisión con profundidad de 5 niveles y métrica de selección de características de la información mutua")
print(cv_results_1)
print("Características seleccionadas después de aplicar SelectFromModel:")
print(X_selected.shape[1])  # Imprimir la cantidad de características seleccionadas



Experimento 1: Árbol de decisión con profundidad de 5 niveles y métrica de selección de características de la información mutua
{'fit_time': array([0.2698791 , 0.27262259, 0.28029323, 0.27657747, 0.27245879]), 'score_time': array([0.00100136, 0.00100064, 0.00050354, 0.00200176, 0.00100303]), 'test_score': array([0.99493671, 0.9964557 , 0.99594937, 0.98784195, 0.9929078 ])}
Características seleccionadas después de aplicar SelectFromModel:
3


<h3><b>Actividad 4: Aplicación a nuevos casos</b></h3>

Introducir texto

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import csv

# Leer el archivo de nuevos casos
datos_nuevos = []
with open('muestra_taller_2_evaluación.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        datos_nuevos.append(row)

# Eliminar filas con valores faltantes
datos_nuevos = np.array(datos_nuevos, dtype='object')
datos_nuevos = datos_nuevos[~np.any(datos_nuevos == '', axis=1)]

# Obtener características de los nuevos casos
X_nuevos = datos_nuevos.astype(float)

# Cargar tus datos de entrenamiento (asegúrate de cambiar 'nombre_del_archivo.csv' por el nombre correcto)
datos_entrenamiento = np.loadtxt('muestra_taller_2_entrenamiento.csv', delimiter=',', skiprows=1)

# Separar características (X) y etiquetas (y)
X_train = datos_entrenamiento[:, :-1]  # Todas las columnas excepto la última
y_train = datos_entrenamiento[:, -1]   # Última columna

# Cargar el modelo mejor encontrado
mejor_modelo = DecisionTreeClassifier(random_state=123, max_depth=5, criterion='entropy', min_samples_split=2, splitter='best')

# Entrenar el modelo con los datos de entrenamiento completos
mejor_modelo.fit(X_train, y_train)

# Obtener las probabilidades de predicción
y_hat = mejor_modelo.predict_proba(X_nuevos)

# Seleccionar 300 casos con mayor probabilidad de ser fraude
indices_inspeccion = np.argsort(y_hat[:, 1])[::-1][:500]

# Generar el archivo "inspeccionar.csv"
with open('inspeccionar.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for index in indices_inspeccion:
        writer.writerow([index])
